## Benchmark for Reading and Datamining PDB Structures with mmtf-pyspark

In [1]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.filters import ContainsGroup
from mmtfPyspark.utils import ColumnarStructure
from mmtfPyspark.interactions import InteractionExtractorPd

import gzip
import pandas as pd
import numpy as np
import os
import time

## Setup the benchmark
Set the path to the MMTF Hadoop Sequence file. Here we retrieve the value of the environment variable MMTF_FULL

path = mmtfReader.get_mmtf_full_path()

Specify a list with the number of cores

cores = [4]

# create results directory
results_dir = '../results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

## Structure Benchmark
This benchmark read structures

def structure(path, num_cores):
    spark = SparkSession.builder.master("local[" + str(num_cores) + "]").appName("Benchmark3").getOrCreate()
    structures = mmtfReader.read_sequence_file(path, first_model=True)                
    count = structures.count()

    spark.stop()
    return count

df_s = pd.DataFrame(columns=('cores', 'structures'))

for num_cores in cores:
    start = time.time()
    count = structure(path, num_cores)
    end = time.time()
    print('structures, cores:', num_cores, 'time:', end-start, 'seconds')
    df_s = df_s.append([{'cores':num_cores, 'structures': end-start, 'count': count}], ignore_index=True, sort=True)

df_s.to_csv(os.path.join(results_dir, 'structures.csv'), index=False)

df_s

In [2]:
spark = SparkSession.builder.master("local[4]").appName("Benchmark3").getOrCreate()
#structures = mmtfReader.read_sequence_file(path, first_model=True)
structures = mmtfReader.download_full_mmtf_files(['4HHB'])
#structures = structures.filter(lambda s: s[0] == '4HHB')
dfs = structures.values().first()

In [3]:
import pandas as pd
# df = pd.DataFrame({'chain_name': dfs.chain_names,
#                                     'chain_id': dfs.chain_ids,
#                                     'group_number': dfs.group_numbers,
#                                     'group_name': dfs.group_names,
#                                     'atom_name': dfs.atom_names,
#                                     'altloc': dfs.alt_loc_list,
#                                     'x': dfs.x_coord_list,
#                                     'y': dfs.y_coord_list,
#                                     'z': dfs.z_coord_list,
#                                     'o': dfs.occupancy_list,
#                                     'b': dfs.b_factor_list,
#                                     'element': dfs.elements,
#                                     'polymer:': dfs.polymer
#                                     })

In [ ]:
#df = pd.DataFrame({'z': dfs.z_coord_list})
#dfs.num_atoms
#dfs.occupancy_list
dfs.z_coord_list

In [ ]:
df.info()


In [ ]:
df['chain_name'] = df['chain_name'].astype('category')
df['group_name'] = df['group_name'].astype('category')
df['atom_name'] = df['atom_name'].astype('category')
df['altloc'] = df['altloc'].astype('category')

In [ ]:
df.info()

In [ ]:
df